In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pydataset import data
import wrangle
import prepare
import math

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler

1) Load the tips dataset.

In [66]:
tips = data('tips')

In [67]:
# split my tips data
tips_train, tips_validate, tips_test = wrangle.split_continuous(tips)

train -> (136, 7)
validate -> (59, 7)
test -> (49, 7)


1a) Create a column named price_per_person. This should be the total bill divided by the party size.



In [68]:
tips_train['price_per_person'] = tips_train['total_bill'] / tips_train['size']


In [69]:
tips_train.head()


,total_bill,tip,sex,smoker,day,time,size,price_per_person
19,16.97,3.50,Female,No,Sun,Dinner,3,5.656667
173,7.25,5.15,Male,Yes,Sun,Dinner,2,3.625000
119,12.43,1.80,Female,No,Thur,Lunch,2,6.215000
29,21.70,4.30,Male,No,Sat,Dinner,2,10.850000
238,32.83,1.17,Male,Yes,Sat,Dinner,2,16.415000


In [70]:
# Split data sets into target and features
X_train = tips_train.drop(columns=['tip'])
y_train = tips_train['tip']
X_validate = tips_validate.drop(columns=['tip'])
y_validate = tips_validate['tip']
X_test = tips_test.drop(columns=['tip'])
y_test = tips_test['tip']

1b) Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

- tip percentage and price per person


1c) Use select k best to select the top 2 features for predicting tip amount. What are they?



In [71]:
X_train_num = X_train.drop(columns=['sex', 'smoker', 'day', 'time'])


In [72]:
X_train_num.head()


,total_bill,size,price_per_person
19,16.97,3,5.656667
173,7.25,2,3.625000
119,12.43,2,6.215000
29,21.70,2,10.850000
238,32.83,2,16.415000


In [73]:
# selecting two most predictive features using SelectKBest

from sklearn.feature_selection import SelectKBest, f_regression

In [76]:
# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(X_train_num, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_num.iloc[:,feature_mask].columns.tolist()

In [77]:
f_feature

['total_bill', 'size']

In [63]:
# selecting two most predictive feature using Recursive Feature Elimination

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 3)

# fit the data using RFE
rfe.fit(X_train_num, y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_num.iloc[:,feature_mask].columns.tolist()

ValueError: Found input variables with inconsistent numbers of samples: [136, 25]

In [42]:
rfe_feature

['total_bill', 'size', 'price_per_person']

1d) Use recursive feature elimination to select the top 2 features for tip amount. What are they?


In [43]:
# Split data sets into target and features
X_train2 = tips_train.drop(columns=['tip_percentage'])
y_train2 = tips_train['tip_percentage']

KeyError: "['tip_percentage'] not found in axis"

In [44]:
X_train_num2 = X_train2.select_dtypes(exclude=['object'])
X_train_num2.head()

NameError: name 'X_train2' is not defined

In [45]:
# selecting two most predictive features using SelectKBest

# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(X_train_num2, y_train2)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_num2.iloc[:,feature_mask].columns.tolist()


NameError: name 'X_train_num2' is not defined

In [46]:
f_feature


['total_bill', 'size', 'price_per_person']

In [47]:
# selecting two most predictive feature using Recursive Feature Elimination

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 3)

# fit the data using RFE
rfe.fit(X_train_num2, y_train2)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_num2.iloc[:,feature_mask].columns.tolist()

NameError: name 'X_train_num2' is not defined

In [48]:
rfe_feature

['total_bill', 'size', 'price_per_person']

1e) Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?


- The iterative nature of the RFE method allows eliminates noise/correlations between features allowing for a more accurate selection process.  Whereas, the KBest selects the top features in one single iteration.  The two methods will tend to converge on the same features as the number of features selected increases.


2) Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [49]:
def select_kbest  (X_df, y_df, n_features):
    '''
    Takes in the predictors, the target, and the number of features to select (k) ,
    and returns the names of the top k selected features based on the SelectKBest class
    
    X_df : the predictors
    y_df : the target
    n_features : he number of features to select (k)
    Example
    select_kbest(X_train_scaled, y_train, 2)
    '''
    
    f_selector = SelectKBest(score_func=f_regression, k= n_features)
    f_selector.fit(X_df, y_df)
    mask = f_selector.get_support()
    X_df.columns[mask]
    top = list(X_df.columns[mask])
    
    return print(f'The top {n_features} selected feautures based on the SelectKBest class are: {top}' )

In [50]:
select_kbest()

TypeError: select_kbest() missing 3 required positional arguments: 'X_df', 'y_df', and 'n_features'

4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [51]:
swiss_df = data('swiss')


In [52]:
data('swiss', show_doc =True)


swiss

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Swiss Fertility and Socioeconomic Indicators (1888) Data

### Description

Standardized fertility measure and socio-economic indicators for each of 47
French-speaking provinces of Switzerland at about 1888.

### Usage

    data(swiss)

### Format

A data frame with 47 observations on 6 variables, each of which is in percent,
i.e., in [0,100].

[,1] Fertility Ig, "common standardized fertility measure" [,2] Agriculture
[,3] Examination nation [,4] Education [,5] Catholic [,6] Infant.Mortality
live births who live less than 1 year.

All variables but 'Fert' give proportions of the population.

### Source

Project "16P5", pages 549-551 in

Mosteller, F. and Tukey, J. W. (1977) “Data Analysis and Regression: A Second
Course in Statistics”. Addison-Wesley, Reading Mass.

indicating their source as "Data used by permission of Franice van de Walle.
Office of Population Research, Princeton Univer

In [53]:
swiss_df.head

<bound method NDFrame.head of               Fertility  Agriculture  Examination  Education  Catholic  \
Courtelary         80.2         17.0           15         12      9.96   
Delemont           83.1         45.1            6          9     84.84   
Franches-Mnt       92.5         39.7            5          5     93.40   
Moutier            85.8         36.5           12          7     33.77   
Neuveville         76.9         43.5           17         15      5.16   
Porrentruy         76.1         35.3            9          7     90.57   
Broye              83.8         70.2           16          7     92.85   
Glane              92.4         67.8           14          8     97.16   
Gruyere            82.4         53.3           12          7     97.67   
Sarine             82.9         45.2           16         13     91.38   
Veveyse            87.1         64.5           14          6     98.61   
Aigle              64.1         62.0           21         12      8.52   
Aubonne 

In [54]:
swiss_df.shape


(47, 6)

In [55]:
#split data in train, validate and split
train, validate, test = wrangle.split_data(swiss_df)

train -> (25, 6)
validate -> (12, 6)
test -> (10, 6)


In [56]:
#split X, y
def split_Xy (train, validate, test, target):
    '''
    This function takes in three dataframe (train, validate, test) and a target  and splits each of the 3 samples
    into a dataframe with independent variables and a series with the dependent, or target variable.
    The function returns 3 dataframes and 3 series:
    X_train (df) & y_train (series), X_validate & y_validate, X_test & y_test.
    '''
    
    #split train
    X_train = train.drop(columns= [target])
    y_train= train[target]
    #split validate
    X_validate = validate.drop(columns= [target])
    y_validate= validate[target]
    #split validate
    X_test = test.drop(columns= [target])
    y_test= test[target]
    return  X_train, y_train, X_validate, y_validate, X_test, y_test
    

In [57]:
#split Xy using my function
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.split_Xy (train, validate, test, 'Fertility' )

X_train -> (25, 5)               y_train->(25,)
X_validate -> (12, 5)         y_validate->(12,) 
X_test -> (10, 5)                  y_test>(10,)


In [58]:
columns = list(X_train.select_dtypes(exclude='object').columns)
columns

['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']

In [59]:
#scaled
X_train_scaled_df, validate_scaled_df, test_scaled_df = prepare.scaled_mimmax(columns, X_train, X_validate, X_test)


In [60]:
#kbest
select_kbest(X_train_scaled_df, y_train, 3)

The top 3 selected feautures based on the SelectKBest class are: ['Examination', 'Catholic', 'Infant.Mortality']


In [61]:
#rfe
select_rfe(X_train_scaled_df, y_train, 3)

NameError: name 'select_rfe' is not defined